### Objetivo: Entender qual é a influência de um blog da nossa marca para que as pessoas venham a comprar nossos produtos em algum momento. 

#### Por: Fabrício Reis de Almeida

### 1- Análise e manipulação do dados

In [1]:
import math

import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
import plotly.offline as py
import plotly.tools as tls
from plotly.offline import plot, iplot, init_notebook_mode
from scipy.stats import stats, normaltest, shapiro, zscore, pearsonr
from scipy import stats

# pio.renderers.default = 'colab'

In [2]:
# Lendo o dataset (csv)
arquivo = 'https://raw.githubusercontent.com/fabriciordalm/datasets/master/blog.csv'
df = pd.read_csv(arquivo)

# Primeiras 5 linhas
df.head()

,data,receita,transacoes_blog,transacoes_site,usuarios_blog,usuarios_site
0,2019-01-01,349184,0,4693,0,46739
1,2019-01-02,867328,0,8501,0,83129
2,2019-01-03,758590,0,9375,0,86067
3,2019-01-04,688873,0,8405,0,69363
4,2019-01-05,441760,0,6136,0,56183


In [3]:
# Ultimas 5 linhas
df.tail()

,data,receita,transacoes_blog,transacoes_site,usuarios_blog,usuarios_site
634,2020-09-26,2509687,1580,15320,9292,96178
635,2020-09-27,2182679,1838,14135,10052,103494
636,2020-09-28,3496475,2684,21724,11224,127524
637,2020-09-29,4519732,3037,28017,11570,150116
638,2020-09-30,6240174,3848,38535,9685,150780


In [4]:
# Quantidade de linhas e colunas
linhas = df.shape[0]
colunas = df.shape[1]

print(f'Linhas: {linhas}')
print(f'Colunas: {colunas}')

Linhas: 639
Colunas: 6


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639 entries, 0 to 638
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   data             639 non-null    object
 1   receita          639 non-null    int64 
 2   transacoes_blog  639 non-null    int64 
 3   transacoes_site  639 non-null    int64 
 4   usuarios_blog    639 non-null    int64 
 5   usuarios_site    639 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 30.1+ KB


In [6]:
df.describe()

,receita,transacoes_blog,transacoes_site,usuarios_blog,usuarios_site
count,6.390000e+02,639.000000,639.000000,639.000000,639.000000
mean,1.623891e+06,528.350548,19039.140845,1439.848200,101610.485133
std,1.160581e+06,1201.781347,13677.729723,3369.872063,37240.234152
min,3.208500e+04,0.000000,3557.000000,0.000000,26298.000000
25%,8.073420e+05,0.000000,11013.000000,0.000000,77727.000000
50%,1.263161e+06,0.000000,16069.000000,0.000000,96104.000000
75%,2.232770e+06,0.000000,22606.500000,0.000000,117586.500000
max,1.226684e+07,5586.000000,188955.000000,13059.000000,369989.000000


In [7]:
df.dtypes

data               object
receita             int64
transacoes_blog     int64
transacoes_site     int64
usuarios_blog       int64
usuarios_site       int64
dtype: object

In [8]:
# Formato: YY/MM/DD
df['data'] = pd.to_datetime(df['data'], yearfirst=True) 

# Removendo os valores negativos da variável receita
df = df[df['receita'] > 0]

### 2- Receitas: Aumento ou Diminuição?
Primeiramente, a pergunta mais lógica seria: A criação do blog tem alguma relação com a receita da empresa? 
Vamos explorar essa hipótese.

In [9]:
# Gráfico -----------------------------------------
fig = px.line(
  df,
  x='data',
  y='receita',
  template='ggplot2',  
  width=800 
)

# Customizações -----------------------------------
anotacoes = []
anotacoes.append(dict(
    xref='paper', yref='paper', x=0.0, y=1.02,
    xanchor='left', yanchor='bottom',
    text='Receita por dia ao longo do tempo',
    font=dict(size=18),
    showarrow=False)
)

fig.update_layout(
    font_family='Fira Sans',
    xaxis_title='Data',
    yaxis_title='Receita',
    annotations=anotacoes,
)

fig.show()

Olhando a evolução da receita ao longo do tempo, dois fatores chamam a atenção:

1. Um grande pico no final de 2019
2. Uma crescente a partir de 2020

In [10]:
print(round(df['receita'].max() - df['receita'].mean()))

10642953


Podemos notar uma grande diferença entre a média e o pico do final de 2019. Possa ser que seja um outlier (dados que se diferenciam drasticamente de todos os outros) mas descartamos quando vemos que se repete em "acesso ao site", por exemplo. Continuando, vamos checar outros possiveis outliers.

In [11]:
fig = px.box(
    df, 
    y='receita',
    template='ggplot2',  
    width=800 
)

fig.show()

In [12]:
xzscore = zscore(df['receita'])
outliers = np.abs(xzscore) > 2.8
outliers_id = np.argwhere([outliers == True])
temp = df 
df_sem_outliers = temp.drop(outliers_id[:,0], axis=0)
outliers = temp.loc[outliers_id[:,0]]

df_sem_outliers = df_sem_outliers[df_sem_outliers['receita'] > 0]

In [13]:
outliers['acimaMedia'] = outliers['receita'].apply(lambda x: x>np.mean(df_sem_outliers['receita']))
outliers[['data', 'receita', 'acimaMedia']].sort_values(by='receita', ascending=False)

,data,receita,acimaMedia
0,2019-01-01,349184,False
0,2019-01-01,349184,False
0,2019-01-01,349184,False
0,2019-01-01,349184,False
0,2019-01-01,349184,False
0,2019-01-01,349184,False
0,2019-01-01,349184,False
0,2019-01-01,349184,False


Levando em consideração que não existe datas comemorativas nas datas dos outliers, e também são todas ocorrências com mais vendas do que a média, é razoável de se supor que essas observações podem ser fruto de promoções, por exemplo. Agora, vamos visualizar as receitas ao longo do tempo novamente, mas sem esses outliers agora:

In [14]:
# Gráfico -----------------------------------------
fig = px.line(
  df_sem_outliers,
  x='data',
  y='receita',
  template='ggplot2',  
  width=800
)

# Customizações -----------------------------------
anotacoes = []
anotacoes.append(dict(
    xref='paper', yref='paper', x=0.0, y=1.02,
    xanchor='left', yanchor='bottom',
    text='Receita por dia ao longo do tempo (sem outliers)',
    font=dict(size=18),
    showarrow=False)
)

fig.update_layout(
    font_family='Fira Sans',
    xaxis_title='Data',
    yaxis_title='Receita',
    annotations=anotacoes,
)

fig.show()

Sem os outliers fica mais fácil ainda de se observar a crescente de receitas em 2020. Além disso, o blog foi criado no em 07/2020, vamos explorar se ele tem algum efeito nessa crescente.

In [15]:
# Gráfico -----------------------------------------
fig = px.histogram(
    df_sem_outliers['receita'],
    template='ggplot2',  
    width=800
)

# Customizações -----------------------------------
anotacoes = []
anotacoes.append(dict(
    xref='paper', yref='paper', x=0.0, y=1.05,
    xanchor='left', yanchor='bottom',
    text='Distribuição da variável receita (sem outliers)',
    font=dict(size=18),
    showarrow=False)
)

fig.update_layout(
    font_family='Fira Sans',
    xaxis_title='Receita',
    yaxis_title='Count',
    annotations=anotacoes,
    # showlegend=False
)

A receita tem certa semelhança com uma distribuição normal. Portanto, vamos fazer um teste de normalidade para termos um melhor embasamento.

In [16]:
t1, p1 = normaltest(df_sem_outliers['receita'])
t2, p2 = shapiro(df_sem_outliers['receita'])

print(t1, p1)
print(t2, p2)

376.1853149456355 2.0530369844243566e-82
0.8357028961181641 4.14442571876992e-25


### 3- Teste de hipóteses

In [17]:
# Data do evento
idx = (df_sem_outliers['usuarios_blog'] > 0).idxmax()  

# Dados antes do evento
X = df_sem_outliers.loc[0:idx-1]['receita']  
nX = len(X)
X_barra = np.mean(X)
S_X = np.var(X)

# Dados após o evento
Y = df_sem_outliers.loc[idx:]['receita']  
nY = len(Y)
Y_barra = np.mean(Y)
S_Y = np.var(Y)

T_obs = (X_barra - Y_barra)/math.sqrt((S_X/ nX) + (S_Y/nY))
numerador = ((S_X/nX) + (S_Y/nY))**2
denominador = ((((S_X/nX)**2)/(nX-1)) + (((S_Y/nY)**2)/(nY-1)))
gl = numerador/denominador

alpha = 0.01
t_critico = -2.343899 # Quantil 

if T_obs <= t_critico:
  print('Hipótese nula rejeitada: a média após o evento é maior')
else: 
    print('Hipótese nula aceita: a média não se alterou')

Hipótese nula rejeitada: a média após o evento é maior


### 4- O blog atrai possíveis compradores?

Outra maneira de avaliar a criação do blog, é entender se as pessoas que passam por ele fazem compras no site. Traduzindo isso para os dados, a pergunta é: Existe correlação entre os acessos ao blog e as transações que passam pelo blog?

In [18]:
# Gráfico -----------------------------------------
fig = px.scatter(
    df,
    x='usuarios_blog', 
    y='transacoes_blog',
    template='ggplot2',  
    width=800
)

# Customizações -----------------------------------
anotacoes = []
anotacoes.append(dict(
    xref='paper', yref='paper', x=0.0, y=1.02,
    xanchor='left', yanchor='bottom',
    text='Correlação entre acessos ao blog e transações vindas do blog',
    font=dict(
        size=18,
        color='rgb(37,37,37)'),
    showarrow=False))

fig.update_layout(
    font_family='Fira Sans',
    xaxis_title='Acessos ao blog',
    yaxis_title='Transações vindas do blog',
    annotations=anotacoes,
)

fig.show()

In [19]:
# Coeficiente de correlação de Pearson
r = pearsonr(df['usuarios_blog'], df['transacoes_blog'])[0]
print(f'r = {r}')

r = 0.8933424151830424


Existe um aumento natural no número de compras feitas por pessoas que passaram pelo blog a medida que os acessos aumentam mas que após um certo tempo, esse número estabiliza. De fato, o blog adquire algum público, que passa a acessá-lo regularmente, mas que não necessariamente sempre faz compras.

### 5- Quais clientes estão acessando o blog: Novos ou antigos?

In [20]:
point1 = [517,-10000]
point2 = [517,175000]
x_values = [point1[0], point2[0]]
y_values = [point1[1], point2[1]]

# Gráfico  -----------------------------------------
fig1 = px.line(
    df_sem_outliers['transacoes_blog'],
    color_discrete_sequence=px.colors.qualitative.G10,
)

fig2 = px.line(
    df_sem_outliers['transacoes_site'],
    color_discrete_sequence=px.colors.qualitative.Light24
)

fig3 = go.Figure(go.Scatter(
    x=x_values,
    y=y_values,
    mode='lines',
    name='Criação do blog'
    )
)

fig4 = go.Figure(data=fig1.data + fig2.data + fig3.data)

# Customizações -----------------------------------
anotacoes = []
anotacoes.append(dict(
    xref='paper', yref='paper', x=0.0, y=1.02,
    xanchor='left', yanchor='bottom',
    text='Compras que não passaram pelo blog',
    font=dict(size=18),
    showarrow=False)
) 

# Legendas
nomes_legendas = ["Compras que passaram pelo blog", "Compras que não passaram pelo blog"]
for idx, name in enumerate(nomes_legendas):
    fig4.data[idx].name = name
    fig4.data[idx].hovertemplate = name

fig4.update_layout(
    font_family='Fira Sans',
    xaxis_title='Dias',
    yaxis_title='Compras',
    template='ggplot2',
    annotations=anotacoes,
    width=1000
)

fig4.show()

Aqui, o objetivo é conseguir entender se o número de pessoas que compram sem passar pelo blog diminuiu após sua criação. Podemos fazer o mesmo procedimento que fizemos com a receita e testar se a média de consumidores que não passam pelo blog diminuiu após sua criação.

Nesse caso vamos fazer uma modificação: Observe que tirando o pico de compras próximo ao final do ano (como vimos na receita), o comportamento dos 4 meses anteriores ao lançamento do blog (a partir do dia 396, para melhor observação no gráfico) é consideravelmente diferente do restante dos meses, apresentando um número maior de compras. Então, nesse teste, vamos comparar os 4 últimos meses antes do blog e o tempo após sua criação (também 4 meses):

In [21]:
xzscore = zscore(df[['transacoes_blog', 'transacoes_site', 'receita']])
outliers = np.abs(xzscore) > 2.8
outliers_id = np.argwhere([outliers == True])
temp = df 
df_sem_outliers = temp.drop(outliers_id[:,0], axis=0)
outliers = temp.loc[outliers_id[:,0]]

In [22]:
# Data do evento
idx = (df_sem_outliers['usuarios_blog'] > 0).idxmax()  

# Dados antes do evento
X = df_sem_outliers.loc[396:idx-1]['transacoes_site'] 
nX = len(X)
X_barra = np.mean(X)
S_X = np.var(X)

# Dados após o evento
Y = df_sem_outliers.loc[idx:]['transacoes_site'] 
nY = len(Y)
Y_barra = np.mean(Y)
S_Y = np.var(Y)

T_obs = (X_barra - Y_barra)/math.sqrt((S_X/ nX) + (S_Y/nY))
numerador = ((S_X/nX) + (S_Y/nY))**2
denominador = ((((S_X/nX)**2)/(nX-1)) + (((S_Y/nY)**2)/(nY-1)))
gl = numerador/denominador

alpha = 0.01
t_critico = 2.599672  # Quantil

if T_obs >= t_critico:
  print('Hipótese nula rejeitada: a média após o evento é menor')
elif T_obs <= -t_critico:
   print('Hipótese nula rejeitada: a média após o evento é maior')
else: 
    print('Hipótese nula aceita: a média não se alterou')

Hipótese nula aceita: a média não se alterou


Em outras palavras, nosso teste aponta que a média de compradores que **não** passaram pelo blog após sua criação não se alterou. Uma hipótese que explica esse comportamento é de que as pessoas que acessam o blog conhecem o site e posteriormente voltam para fazer compras direto no site, não necessariamente passando pelo blog. Porém, por outro lado, esse pode ser um indicativo de que o blog não está sendo muito efetivo com antigos clientes, confirmando a hipótese de que os acessos ao blog são de **novos clientes**.

### 6- Conclusão

Após esses tópicos, temos evidências bem claras que **a influência do blog para a empresa é positiva**.

# 7- Referências
- https://pandas.pydata.org/docs/reference
- https://numpy.org/doc/stable/reference
- https://plotly.com/python
- https://plotly.com/python-api-reference/
- https://www.scribbr.com/statistics/pearson-correlation-coefficient/#:~:text=The%20Pearson%20correlation%20coefficient%20